In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


# Get all foodpanda orders
def get_foodpanda_orders(orders, cookie):

    url = "https://www.foodpanda.sg/orders"

    headers = {
    #Insert cookies
    }

    resp = requests.get('https://www.foodpanda.sg/orders', headers=headers)
    html = resp.content

    soup = BeautifulSoup(html, 'html.parser')

    order_entry = soup.findAll('li')

    order_entry = list(set(order_entry))

    
    for order in order_entry:
        if order is not None:

            order_plc_date = order.find("span", {"class": "vendor-name"})
            print(order_plc_date)
            print(orders)
            
            if order_plc_date is not None:
                vendor = order_plc_date.getText().strip().replace("\n", "").split(",", 1)[0]

                order_date = order.find("div", {"class": "order-date"}).getText().strip()
                order_date = datetime.datetime.strptime(order_date, "%d %b, %Y")
            else:
                continue

            bill = order.find("span", {"class": "total"})

            if bill is not None:
                print(bill.getText())
                cost = bill.getText().strip()

            contents = order.find("div", {"class": "order-product-list"})

            if contents is not None:
                food = contents.getText().split("\n")
                items = [i.strip() for i in food if i.strip()]

            orders.append({'Restaurant': vendor, 'Date': order_date,
                           'Cost': cost, 'Items': items,
                           'Service': 'foodpanda'})

    return orders





In [ ]:
# Store all orders in a list
orders = []

# Get foodpanda order for all users
foodpanda_orders = get_foodpanda_orders(orders, [])

# Convert orders to pandas dataframe
df = pd.DataFrame(orders)

# Export dataframe into csv file
df.to_csv("foodpanda_orders.csv", index=False)